# Phân loại sắc thái bình luận bằng mô hình CNN

## 1. Dữ liệu (Data)

Nguồn: Cuộc thi phân loại sắc thái bình luận của aivivn `https://www.aivivn.com/contests/1`

Mô tả: 3(file): file_train: train.crash, file test: test.crash, sample_submission.csv.

Dữ liệu: Câu bình luận có độ dài bất kỳ được gán nhãn 0 hoặc 1, trong đó 1: bình luận tiêu cực, 0: bình luận tích cực

train.crash: 16086, test.crash: 10980



## 2. Tổng quan về phương pháp

Bước 1. Tiền xử lý dữ liệu: lấy được các trường dữ liệu: `id, label, review`

Bước 2. Sử dụng `FastText` để `Word Embedding`: biểu diễn mỗi từ là vector có 50 phần tử.

Bước 3. Chuẩn hóa độ dài các câu trong dữ liệu train `len(max(review)) = 679`. Thêm các từ viền PAD.

Bước 4. Đưa vào mô hình CNN: `Conv1D`, `optimizer='adam'`.

Bước 5. Predict với dữ liệu `validation_data`.

Bước 6. Predict với dữ liệu `test`.

Bước 7. Đánh giá độ chính xác và kết luận.

## 3. Xây dựng mô hình

### Lấy các trường dữ liệu

In [1]:
def _create_row(sample):        
        d = {}
        d['id'] = sample[0].replace('\n','')
        review = ""

        for clause in sample[1:-1]:
            review+= clause.replace('\n','').strip()
        d['label'] = int(sample[-1].replace('\n',''))
            
        d['review'] = review
        return d
    
def _load_raw_data():
    a = []
    b = []
    with open('./data/train.crash', 'r', encoding='utf8') as f:
        for line in f:
            if 'train_' in line:
                b.append(a)
                a = [line]  
            elif line!='\n':
                a.append(line) 
        b.append(a)
    return b[1:]

In [2]:
raw_data = _load_raw_data()
raw_data[0:3]

[['train_000000\n',
  '"Dung dc sp tot cam on \n',
  'shop Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời"\n',
  '0\n'],
 ['train_000001\n',
  '" Chất lượng sản phẩm tuyệt vời . Son mịn nhưng khi đánh lên không như màu trên ảnh"\n',
  '0\n'],
 ['train_000002\n',
  '" Chất lượng sản phẩm tuyệt vời nhưng k có hộp k có dây giày đen k có tất"\n',
  '0\n']]

In [3]:
lst = []
for row in raw_data:
    lst.append(_create_row(row))

In [4]:
lst[1:3]

[{'id': 'train_000001',
  'label': 0,
  'review': '" Chất lượng sản phẩm tuyệt vời . Son mịn nhưng khi đánh lên không như màu trên ảnh"'},
 {'id': 'train_000002',
  'label': 0,
  'review': '" Chất lượng sản phẩm tuyệt vời nhưng k có hộp k có dây giày đen k có tất"'}]

### Token các từ trong bình luận

In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
X_train = []
y_train = []
for l in lst:
    tokens_line = nltk.word_tokenize(l['review'][1:-2].lower())
    X_train.append(tokens_line)
    y_train.append([l['label']])

In [8]:
X_train[0:3]

[['dung',
  'dc',
  'sp',
  'tot',
  'cam',
  'onshop',
  'đóng',
  'gói',
  'sản',
  'phẩm',
  'rất',
  'đẹp',
  'và',
  'chắc',
  'chắn',
  'chất',
  'lượng',
  'sản',
  'phẩm',
  'tuyệt',
  'vờ'],
 ['chất',
  'lượng',
  'sản',
  'phẩm',
  'tuyệt',
  'vời',
  '.',
  'son',
  'mịn',
  'nhưng',
  'khi',
  'đánh',
  'lên',
  'không',
  'như',
  'màu',
  'trên',
  'ản'],
 ['chất',
  'lượng',
  'sản',
  'phẩm',
  'tuyệt',
  'vời',
  'nhưng',
  'k',
  'có',
  'hộp',
  'k',
  'có',
  'dây',
  'giày',
  'đen',
  'k',
  'có',
  'tấ']]

In [9]:
y_train[0:3]

[[0], [0], [0]]

### Biểu diễn các từ thành các vector độ dài 50

In [10]:
from gensim.models import FastText
num_features = 50 # số phần tử vector từ để biểu diễn từ
model = FastText(X_train, size = num_features)

In [11]:
model.wv.similar_by_word("tốt")

[('tốt..', 0.9387295246124268),
 ('tố', 0.9257695078849792),
 ('tốtrất', 0.8866126537322998),
 ('tốtthời', 0.8539071083068848),
 ('chắn.shop', 0.8306227326393127),
 ('vụ', 0.8274441957473755),
 ('chắnshop', 0.8272778391838074),
 ('chắnthời', 0.7971609234809875),
 ('chắnrất', 0.7832304239273071),
 ('nhanh.shop', 0.7824429869651794)]

In [12]:
model.wv["tốt"]

array([ 0.7861665 ,  0.3493711 , -1.4547497 ,  0.30683073,  0.2967116 ,
       -0.15043758, -0.49263048, -2.4581242 ,  0.0238688 ,  0.50196725,
       -1.6451759 , -0.03142986,  1.5748236 ,  0.8722173 ,  0.59551466,
        1.3626158 ,  0.8051109 , -1.0753506 ,  0.35939068,  0.52111614,
        0.11620174,  0.0551413 ,  1.5276858 ,  0.0726677 , -0.17386746,
        1.0182841 , -0.58345735,  0.24316444, -0.02629043, -0.04641446,
        0.4349537 ,  0.72265255,  0.4430185 , -0.1334187 ,  0.41484886,
        1.5819662 ,  1.8601044 ,  0.6961471 , -1.1983576 ,  0.7633791 ,
       -1.141518  ,  0.82182556,  0.87123686,  0.93413454,  2.6408226 ,
       -0.2873781 ,  2.1390705 ,  2.4530299 ,  0.21077135,  0.7954732 ],
      dtype=float32)

In [13]:
len_max_sen = max([len(x) for x in X_train])
len_max_sen

679

Độ dài câu dài nhất trong X_train là `679`. Chúng ta sẽ coi mỗi câu có độ dài như nhau là 679 (để đưa vào mô hình được thì phải thống nhất kích thước). Mỗi từ được biểu diễn qua model FastText đã train, với những câu không đủ độ dài 679, chúng ta thêm các từ `PAD` và coi đó là từ viền cho 1 câu.

###  Thêm các từ PAD: từ viền cho 1 câu.

In [14]:
X_train_num = []
for sent in X_train:
    temp = sent
    # thêm PAD
    if len(sent) < len_max_sen:
        add_element = len_max_sen - len(sent)
        for _ in range(add_element):
            temp.append('PAD')
    # vector hoá
    for i in range(len(sent)):
        sent[i] = model.wv[sent[i]]
    X_train_num.append(temp)

In [15]:
import numpy as np
# Lấy khoảng 10k câu để train
X_train_num = np.array(X_train_num[:10000])
y_train_num = np.array(y_train[:10000])

In [16]:
X_train_num.shape

(10000, 679, 50)

In [17]:
y_train_num.shape

(10000, 1)

### Model CNN

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten

maxlen = 679
batch_size = 32
embedding_dims = 50
filters = 32
kernel_size = 3
hidden_dims = 250

CNN = Sequential()
CNN.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1, input_shape=(maxlen, embedding_dims)))
CNN.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
CNN.add(Flatten())
CNN.add(Dense(hidden_dims, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(1, activation='sigmoid'))
CNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
CNN.summary()

Using TensorFlow backend.
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 677, 32)           4832      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 675, 32)           3104      
_________________________________________________________________
flatten_1 (Flatten)          (None, 21600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               5400250   
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 5,408,437
Tr

In [19]:
X_val = np.array(X_train[10000:])
y_val = np.array(y_train[10000:])

In [20]:
epochs = 5
CNN.fit(X_train_num ,y_train_num, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))


Train on 10000 samples, validate on 6087 samples
Epoch 1/5
10000/10000 [==============================] - 75s 8ms/step - loss: 0.3689 - accuracy: 0.8311 - val_loss: 0.3182 - val_accuracy: 0.8630
Epoch 2/5
10000/10000 [==============================] - 61s 6ms/step - loss: 0.2975 - accuracy: 0.8713 - val_loss: 0.3397 - val_accuracy: 0.8543
Epoch 3/5
10000/10000 [==============================] - 63s 6ms/step - loss: 0.2640 - accuracy: 0.8889 - val_loss: 0.3197 - val_accuracy: 0.8643
Epoch 4/5
10000/10000 [==============================] - 60s 6ms/step - loss: 0.2320 - accuracy: 0.9002 - val_loss: 0.3246 - val_accuracy: 0.8671
Epoch 5/5
10000/10000 [==============================] - 56s 6ms/step - loss: 0.2047 - accuracy: 0.9132 - val_loss: 0.3541 - val_accuracy: 0.8668


In [21]:
CNN.save("model.h")

In [22]:
score = CNN.evaluate(X_val, y_val, verbose=0)

In [23]:
print(score)

[0.3540663365735203, 0.8667652606964111]
